### Initialization

In [1]:
"""IMPORTS"""

from arcgis.gis import GIS
from arcgis.gis import Item
from arcgis.gis import ContentManager
from arcgis import __version__
from arcgis.mapping.ogc import CSVLayer
from io import BytesIO
import ipywidgets as widgets

import pandas as pd
import tempfile

import os
import uuid
import json
import shutil
import tempfile

from getpass import getpass

In [2]:
"""CONSTANTS""" 

### DO NOT CHANGE

ITEM_COPY_PROPERTIES = ['title', 'type', 'typeKeywords', 'description', 'tags',
                        'snippet', 'extent', 'spatialReference', 'name',
                        'accessInformation', 'licenseInfo', 'culture', 'url']

TEXT_BASED_ITEM_TYPES = frozenset(['Web Map', 'Feature Service', 'Map Service','Web Scene', 'Dashboard',
                                   'Image Service', 'Feature Collection', 
                                   'Feature Collection Template',
                                   'Web Mapping Application', 'Mobile Application', 
                                   'Symbol Set', 'Color Set',
                                   'Windows Viewer Configuration'])

FILE_BASED_ITEM_TYPES = frozenset(['File Geodatabase','CSV', 'Image', 'KML', 'Locator Package',
                                  'Map Document', 'Shapefile', 'Microsoft Word', 'PDF',
                                  'Microsoft Powerpoint', 'Microsoft Excel', 'Layer Package',
                                  'Mobile Map Package', 'Geoprocessing Package', 'Scene Package',
                                  'Tile Package', 'Vector Tile Package'])

RELATIONSHIP_TYPES = frozenset(['Map2Service', 'WMA2Code',
                                'Map2FeatureCollection', 'MobileApp2Code', 'Service2Data',
                                'Service2Service'])

_version = [int(i) for i in __version__.split('.')]


### CONFIGURE HERE

In [5]:
### HARDCODED CONFIG FOR TESTING SM TRANSFER

origin_username = "rfreed_manucomm"
origin_pass = getpass(prompt="Enter the password associated with your named user (Not your SSO credentials!): ")

# Establish origin and target GIS organizations
print("Connecting ...")
origin = GIS("https://manucomm.maps.arcgis.com/", origin_username, origin_pass, expiration=9999)
print("Connection Successful.")

destination = GIS("home", expiration=9999)

catalog = destination.content.get("d7c78bf632264b148cc7c45ebfaf489d")
catalog_table = pd.DataFrame.spatial.from_layer(catalog.tables[0])
catalog_table.drop(columns=['ObjectId'])


Enter the password associated with your named user (Not your SSO credentials!): ········
Connecting ...
Connection Successful.


/opt/conda/lib/python3.9/site-packages/arcgis/gis/__init__.py:676: UserWarning: You are logged on as rya12533@esri.com_esri_cgs with an administrator role, proceed with caution.
  warnings.warn(


In [9]:
# get items for the current user
my_username = origin.users.me.username
me = origin.users.search(my_username)[0]
my_items = me.items()

# origin items by title
origin_items_by_title = [item for item in my_items if item.title != "catalog"]

item_select = widgets.SelectMultiple(
    options=origin_items_by_title,
    description="Items: ",
    disabled=False
)

print(f"Select the items you would like to transfer from the selection box below: ")
item_select

Select the items you would like to transfer from the selection box below: 


SelectMultiple(description='Items: ', options=(), value=())

### Helper Functions

In [66]:
def rebuild_relations(destination: GIS, relation_map: dict):
    """
    Helper Function
    
    Applies origin item relations to the destination portal item.
    
    Arguments:
        destination (arcgis.gis.GIS): Destination GIS for the origin item.
        relation_map (dict): Relation map returned from wc_transfer() that maps the origin and destination IDs. 
    """
    
    # traverse all keys in the relation map, representing the source item IDs
    for key in relation_map.keys():
        origin_item = origin_items_by_id[key]
        destination_itemid = relation_map[key]
        destination_item = destination.content.get(destination_itemid)
        
        # find the relations attatched to this ID 
        for relationship in RELATIONSHIP_TYPES:
            try:
                origin_related_items = origin_item.related_items(relationship)
                
                # apply relationships to target items 
                for origin_related_item in origin_related_items:
                    destination_related_itemid = relation_map[origin_related_item.itemid]
                    destination_related_item = destination.content.get(destination_related_itemid)
                    status = destination_item.add_relationship(destination_related_item, relationship)
                    print(f"After execution, relation type {relation} between {destination_item.title} and {destination_related_item.title} is {status}.")
            except Exception as rel_ex: # bare except clause temporary
                print(f"Error when checking for {relationship}: {rel_ex}")
                continue
    
    print("Process complete. Please check your content on the destination portal and review errors raised in this notebook.")
    
def export_resources(item, save_path=None, file_name=None):
    """
    Helper function, from https://developers.arcgis.com/python/samples/clone-storymap-version2/
    Export's the data's resources as a zip file
    """
    
    url = f'{item._gis._portal.resturl}content/users/{item._user_id}/items/{item.itemid}/resources/export'
    if save_path is None:
        save_path = tempfile.gettempdir()
    if file_name is None:
        file_name = f"{uuid.uuid4().hex[:6]}.zip"
    params = {'f' : 'zip'}
    con = item._gis._portal.con
    resources = con.get(url, params=params,
                        out_folder=save_path,
                        file_name=file_name,
                        try_json=False)
    return resources


### DEPRECATED
"""
def copy_item(destination: GIS, origin_item: Item):
    
    Helper function
    
    Creates an item in the destination org to be populated with data via wc_transfer(). 
    
    Params:
        destination (arcgis.gis.GIS): Destination GIS for the origin item.
        origin_item (arcgis.gis.Item): Item to be copied by the GIS
        
    try:
        # helpfile to read origin item content and write it to memory
        with tempfile.TemporaryDirectory() as tempdir:
            item_properties = {}
            # write the origin property to the item property dictionary
            for property_name in ITEM_COPY_PROPERTIES:
                item_properties[property_name] = origin_item[property_name]
            
            data_file = None
            
            # if its a text based item, just use the get_data function
            if origin_item.type in TEXT_BASED_ITEM_TYPES:
                text = origin_item.get_data(False)
                item_properties['text'] = text
            
            # if file based, download the content to a temporary directory
            elif origin_item.type in FILE_BASED_ITEM_TYPES:
                data_file = origin_item.download(tempdir)
            
            thumbnail_file = origin_item.download_thumbnail(tempdir)
            metadata_file = origin_item.download_metadata(tempdir)
            
            destination_item = destination.content.add(item_properties, data_file, thumbnail_file, metadata_file, origin_item.owner)
            
            # Sharing configuration
            destination_item.share(origin_item.access)
    except Exception as copy_ex: # Bare except clause for now, will change later
        # TODO
        print(f"Error Copying over origin item: {copy_ex}")
"""
        
def get_layer_item_ids(wm):
    """
    Helper function from https://developers.arcgis.com/python/guide/cloning-content/
    
    Returns the related items in a webmap.
    
    Params:
        wm (argis.gis.Item): Webmap item to be inspected.
    Returns:
        wm_id_list (list): List of related items in the web map.
    """
    wmo = Webmap(wm)
    wm_id_list = []
    
    for layer in wmo.layers:
        try:
            fsvc = FeatureLayerCollection(layer['url'][:-1], origin)
            if not fsvc.properties['serviceItemId'] in wm_id_list:
                wm_id_list.append(fsvc.properties['serviceItemId'])
        except Exception as e:
            pass
    return wm_id_list
    

def iterate_all(iterable, returned="key"):
    # Credits: https://gist.github.com/PatrikHlobil/9d045e43fe44df2d5fd8b570f9fd78cc
    
    """Returns an iterator that returns all keys or values
       of a (nested) iterable.
       
       Arguments:
           - iterable: <list> or <dictionary>
           - returned: <string> "key" or "value"
           
       Returns:
           - <iterator>
    """
  
    if isinstance(iterable, dict):
        for key, value in iterable.items():
            if returned == "key":
                yield key
            elif returned == "value":
                if not (isinstance(value, dict) or isinstance(value, list)):
                    yield value
            else:
                raise ValueError("'returned' keyword only accepts 'key' or 'value'.")
            for ret in iterate_all(value, returned=returned):
                yield ret
    elif isinstance(iterable, list):
        for el in iterable:
            for ret in iterate_all(el, returned=returned):
                yield ret
                
def find_relates(item: Item, gis=GIS("home")):
    """
    Finds the AGOL items nested inside an item without needing an Enterprise platform or 
    manually specifying relationships using the ArcGIS API. Does not tell you the type of 
    relationship between items, only the ItemIDs associated with a given item.
    
    Arguments:
        gis (GIS): GIS object from arcgis.gis.GIS that the item lives in.
        item (Item): Item object from arcgis.gis.Item
        
    Returns:
            related_ids (set): All unique item IDs related to an item.
    """
    
    related_ids = []
    item_json = item.get_data(try_json=True)
    formatted_keys = iterate_all(item_json, returned="value")
    
    json_vals = list(formatted_keys)
    json_vals = [val for val in json_vals if isinstance(val, str)]
    
    for val in json_vals:
        try:
            search = gis.content.get(val)
            if search != None:
                related_ids.append(val)
        except:
            pass
    
    return set(related_ids)

SyntaxError: invalid syntax (2727330710.py, line 117)

### Transfer Functions

In [62]:
def dash_transfer(destination: GIS, dash):
    """
    TODO
    
    Performs a web content transfer of a dashboard to destination AGOL.
    
    Arguments:
        destination (GIS): Destination GIS for the given Dashboard
        dash (Item): A Dashboard item in the destination GIS
    """
    
    # PSEUDO CODE!
    wm_items = {}
    dash_elements = find_relates(dash)

    for ele in dash_elements:
        entry = catalog_table.loc[catalog_table['source_id'] == ele]
        wm_items[ele] = entry['destination_id']
        if entry is None:
            print("Dependency has not been cloned yet. Aborting.")
            exit()
    
    destination.content.clone_items(items=[dash], item_mapping=wm_items)
    print(f"Clone successful. Refresh your content page.")

def wc_transfer(destination: GIS, items=[]):
    """
    Performs a web content transfer of items from an origin to destination AGOL. 
    
    Arguments:
        destination (arcgis.gis.GIS): Destination GIS for the given items
        items (list): a list of Items to be transferred.
    """
    
    origin_to_destination_ids = {}
    
    # check if an item has been transferred. if it has, remove it from the clone list.
    for item in items:
        if item.id in catalog_table['source_id'].unique():
            items.remove(item)
            print(f"{item.title} already transferred. Removing it from the queue...")
    
    destination_items = destination.content.clone_items(items)
    
    # build origin to destination map for rebuild_relations function and to memoize to catalog
    origin_item_index = 0
    for destination_item in destination_items:
        if destination_item:
            origin_to_destination_ids[item.id] = destination_item.id
        else:
            origin_to_destination_ids[item.id] = None
            
        new_record = pd.DataFrame({"source_id": item[origin_item_index],
                           "destination_id": destination_item.itemid,
                           "title": destination_item.title,
                           "owner": destination_item.owner})
        catalog_table = pd.concat([catalog_table, new_record])
        origin_item_index += 1
    
    # export to csv in memory, and overwrite current catalog feature layer
    catalog_table.to_csv('catalog.csv')
    
    collection = FeatureLayerCollection.fromitem(catalog)
    collection.manager.overwrite('catalog.csv')
        
    # run rebuild helper function
    rebuild_relations(destination=destination, relation_map=origin_to_destination_ids)

def sm_transfer(destination: GIS, item: Item):
    """
    Adapted code sample from https://developers.arcgis.com/python/samples/clone-storymap-version2/
    
    Transfer protocol for Story Maps and their web content items. Does not call copy_items() as protocol is different for this content.
    
    Arguments:
        destination (arcgis.gis.GIS): Destination GIS for the origin item.
    """

    if item.id in catalog_table['source_id'].unique():
        print(f"Story Map with ID {item.id} has already been transferred. Moving to next portal item.")
        pass
    story_map = item

    # check version to apply relevant protocol
    if _version <= [1, 8, 2]:
        resource = export_resources(item=story_map)
    else:
        resource = story_map.resources.export()

    # get story map item data from json to store related maps
    story_map_json = story_map.get_data(try_json=True)

    web_maps = set([v['data']['itemId'] for k, v in story_map_json['resources'].items() \
            if v['type'].lower().find('webmap')>-1])
    express_maps = set([v['data']['itemId'] for k, v in story_map_json['resources'].items() \
            if v['type'].lower().find('expressmap')>-1])


    webmap_mapper = {} # keys are origin IDs, values are destination IDs
    for wm in web_maps:
        webmap_to_copy = origin.content.get(wm)

        # check if item has been tranferred, if no then duplicate, if yes point to content that already exists
        if webmap_to_copy.id in catalog_table['source_id'].unique():
            webmap_destination_id = catalog_table.loc[catalog_table['source_id'] == webmap_to_copy.id, 'destination_id']
            webmap_mapper[webmap_to_copy.id] = webmap_destination_id
        else:
            cloned_webmaps = destination.content.clone_items([webmap_to_copy])
            webmap_mapper[webmap_to_copy.id] = [i for i in cloned_webmaps if i.type == 'Web Map'][0].id

            # memoize tranfer to catalog
            new_record = pd.DataFrame({"source_id": [webmap_to_copy.id],
                                       "destination_id": [webmap_mapper[webmap_to_copy.id]], # gets destination ID from corresponding origin ID
                                       "title": [webmap_to_copy.title],
                                       "owner": [webmap_to_copy.owner]})
            catalog_table = pd.concat([catalog_table, new_record])
    
    # export to csv in memory, and overwrite current catalog feature layer
    catalog_table.to_csv('catalog.csv')
    
    collection = FeatureLayerCollection.fromitem(catalog)
    collection.manager.overwrite('catalog.csv')

    # remap the old itemid to the new one
    story_map_text = json.dumps(story_map_json)

    for key, val in webmap_mapper.items():
        story_map_text = story_map_text.replace(key, val)

    new_item = destination.content.add({'type' : story_map.type,
                             'tags' : story_map.tags,
                             'title' : story_map.title,
                             'description' : story_map.description,
                             'typeKeywords' : story_map.typeKeywords,
                             'extent' : story_map.extent,
                             'text' :story_map_text}
                            )

    # bring in the storymap resources exported to a zip archive earlier
    new_item.resources.add(resource, archive=True)

    # update the url
    new_item.update({'url': story_map.url.replace(story_map.id, new_item.id)})
    print(f"StoryMap transfer complete. You can visit it at {new_item.homepage}")

def transfer(items=item_select.value):
    """
    Runs the appropriate transfer function for each item the user has chosen from the select widget.
    
    Arguments:
        items (iterable): An iterable of Item IDs. Default is the set generated from the selection box in the Content Migration Notebook.
    """
    
    sm_items = []
    wc_items = []
    dsh_items = []
    
    # partition items from selection into modern Storymap workflow and web content workflow
    for item in items:
        if item.type == "StoryMap":
            sm_items.append(item)
        elif item.type == "Dashboard":
            dsh_items.append(item)
        else:
            wc_items.append(item)
    
    # story map transfer protocol
    for item in sm_items:
        sm_transfer(destination=destination, item=item)
    
    for item in dsh_items:
        dash_transfer(destination=destination, items=dsh_items)
    
    # web content transfer protocol
    wc_transfer(destination=destination, items=sm_items)

In [63]:
transfer()

AttributeError: 'list' object has no attribute 'id'